In [1]:
%reload_ext setup

INFO:setup:IPython environment configured successfully!


## Introduction to Agents

In [ ]:
# Cell 1: Imports and Logging Configuration
import logging
from igent.matching_agent import get_agent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

# Suppress autogen_core and httpx INFO logs
logging.getLogger("autogen_core").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# Cell 2: Get the agent
agents = await get_agent(state_path="model_config.yaml")
# display(agents)

# Cell 3: Run the chat with simplified output
message = "Hello, tell a joke?"
async for msg in agents.run_stream(
    task=[TextMessage(content=message, source="user")],
    cancellation_token=CancellationToken()
):
    if isinstance(msg, TextMessage):  # Only show actual text messages between agents
        print(f"{msg.source}: {msg.content}")
    elif isinstance(msg, TaskResult):  # Show final result
        final_message = "Task completed. "
        if msg.stop_reason:
            final_message += f"Stop reason: {msg.stop_reason}"
        print(final_message)

user: Hello, tell a joke?
matcher: Sure! Why don't scientists trust atoms?

Because they make up everything!
critic: This joke is a classic play on words with a clever twist. It's simple and somewhat expected, but it's a great introduction to scientific humor. To enhance your joke-telling, you might consider tailoring the joke to align with current trends or specific interests of the person you're sharing it with, making it feel more personal and engaging. If you have any other jokes or need help tweaking a joke to fit an occasion, feel free to ask!
matcher: Thank you for the feedback! I appreciate your insight. Let me know if you'd like to hear another joke or if you have a specific request.
critic: APPROVE
Task completed. Stop reason: Text 'APPROVE' mentioned


## Demo Example (20 registrants)

In [ ]:
general_context = """
<general_context>
iChoosr helps match potential customers with vetted suppliers on a large scale. They help them choose sustainable energy solutions including solar panels, home batteries, heat pumps, and renewable energy tariffs. They operate globally in Belgium, Japan, the Netherlands, the United Kingdom, the United States, and more. They try to balance both sides of the transaction between the registrant and suppliers by making the process efficient for both parties. Generally, the process of matching the registration to suppliers and the eligible subsidies looks like this:
- Step 1: Match the registration to the correct supplier based on <matching_rules>
- Step 2: Validate the output of the matching expert based on <matching_rules>.\
- Step 3: Enrich the matching output with the right product price and select the eligible subsidies that apply for this registration based on the <subsidy_rules>.
- Step 4: Validate the output of the subsidy and price expert based on the <subsidy_rules>.
- Step 5: Generate the final JSON based on the validated matching outcome.
</general_context>
"""
matcher_role = """
<role_matching_expert>
You are a matching expert responsible for efficiently matching registrations with suppliers based on their ZIP code and requested sustainable energy solutions.
Your goal is to match the registration to the correct supplier based on <matching_rules>.

The data from a registration point of view will come from the registration data and the supplier data will come from the auction outcomes dataset. Your task is to process registrations, query the supplier offer dataset, and return the best possible match while considering registration and supplier location, and supplier capacity constraints. Ensure you always adhere to the matching rules as defined in <matching_rules>. 
</role_matching_expert>
"""
critic_role = """
<role_matching_critic>””You are an expert in examining and validating the output of the <role_matching_expert> and you always ensure that every match between registration and supplier is correct and meets the <matching_rules.txt> consistently. The output data of the <role_matching_expert> is always based on the <matching_rules.txt> and you will receive it in JSON format. Your task is to validate if the output data is correct based on the defined rules in <matching_rules.txt>.

Your task is to validate the output data of the <role_matching_expert>, guaranteeing the best possible match while considering registration and supplier location, and supplier capacity constraints.

To summarise, you are responsible for step 2 of the matching process found in <ichoosr_matching_context>””. Follow these steps in order to validate the matching:

1. Check if the output data is correct based on the <matching_rules.txt>.
2a. If data is correct, generate final JSON based on validated matching data and Terminate.
2b. If data is incorrect, return data to <role_matching_expert> highlighting the reason why the matching is done wrong and feedback on how to approach it better. Continue this loop until the data is correct, or loop has repeated up to 5 times. If data remains incorrect after 5 tries, submit ‘review request from a human’.
"""